# Import Libraries

In [2]:
import time
import json
from pymongo import MongoClient
import GetOldTweets3 as got
import pandas as pd
import matplotlib.pyplot as plt
import string 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import warnings
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
import spacy
nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#HATE TWEETS: https://towardsdatascience.com/detecting-hate-tweets-twitter-sentiment-analysis-780d8a82d4f6

# WEB SCRAPING

In [3]:
# Social Media Scrapping - TWITTER:
dates = ["2020-05-22","2020-05-23","2020-05-24","2020-05-25","2020-05-26","2020-05-27","2020-05-28","2020-05-29","2020-05-30",
        "2020-05-31","2020-06-01","2020-06-02","2020-06-03","2020-06-04","2020-06-05","2020-06-06","2020-06-07","2020-06-08",
        "2020-06-09","2020-06-10", "2020-06-11","2020-06-12","2020-06-13","2020-06-14","2020-06-15","2020-06-16","2020-06-17",
        "2020-06-18","2020-06-19","2020-06-20","2020-06-21"]

total_tweets = []

for i in range(1,len(dates)):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch('#ACAB')\
                                               .setSince(str(dates[i-1]))\
                                               .setUntil(str(dates[i]))\
                                               .setMaxTweets(10000)\
                                               .setLang('en')
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    
    total_tweets.extend(tweets)                                              
                                                                                                               
text_tweets_acab = [[tw.id,
                    tw.username,
                    tw.text,
                    tw.date,
                    tw.favorites,
                    tw.retweets,
                    tw.mentions,
                    tw.hashtags] for tw in total_tweets]
                                                       
df_tweets_acab = pd.DataFrame(text_tweets_acab, columns = ['ID', 'User', 'Text', 'Date', 'Favorites', 'Retweets', 'Mentions','Hashtags'])

In [4]:
# Social Media Scrapping - TWITTER:
dates = ["2020-05-22","2020-05-23","2020-05-24","2020-05-25","2020-05-26","2020-05-27","2020-05-28","2020-05-29","2020-05-30",
        "2020-05-31","2020-06-01","2020-06-02","2020-06-03","2020-06-04","2020-06-05","2020-06-06","2020-06-07","2020-06-08",
        "2020-06-09","2020-06-10", "2020-06-11","2020-06-12","2020-06-13","2020-06-14","2020-06-15","2020-06-16","2020-06-17",
        "2020-06-18","2020-06-19","2020-06-20","2020-06-21"]

total_tweets = []

for i in range(1,len(dates)):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch('#blacklivesmatter')\
                                               .setSince(str(dates[i-1]))\
                                               .setUntil(str(dates[i]))\
                                               .setMaxTweets(10000)\
                                               .setLang('en')
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    
    total_tweets.extend(tweets)                                              
                                                                                                               
text_tweets_blm = [[tw.id,
                    tw.username,
                    tw.text,
                    tw.date,
                    tw.favorites,
                    tw.retweets,
                    tw.mentions,
                    tw.hashtags] for tw in total_tweets]
                                                       
df_tweets_blm = pd.DataFrame(text_tweets_blm, columns = ['ID', 'User', 'Text', 'Date', 'Favorites', 'Retweets', 'Mentions','Hashtags'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: <urlopen error [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder>
Try to open in browser: https://twitter.com/search?q=%23blacklivesmatter%20since%3A2020-06-15%20until%3A2020-06-16&src=typd
Traceback (most recent call last):
  File "C:\Users\David\anaconda3\envs\Sentiment\lib\urllib\request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "C:\Users\David\anaconda3\envs\Sentiment\lib\http\client.py", line 1262, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\David\anaconda3\envs\Sentiment\lib\http\client.py", line 1308, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\Users\David\anaconda3\envs\Sentiment\lib\http\client.py", lin

TypeError: must be str, not list

In [5]:
len(df_tweets_acab.User)

110631

In [11]:
len(df_tweets_blm.User)

202572

In [7]:
# Safe Data in DATABASE:
MONGO_HOST= 'mongodb://localhost/'
client = MongoClient(MONGO_HOST)
try:
    connect = MongoClient()
    print("Connected successfully!!!")
except:
    print("Could not connect to MongoDB")
    
db = connect.Hashtags_all
collection = db.Final_ACAB

for i in range(0,len(df_tweets_acab.User)):
    item = {
            "ID": str(df_tweets_acab.ID[i]),
            "User": str(df_tweets_acab.User[i]),
            "Text": str(df_tweets_acab.Text[i]),
            "Date": str(df_tweets_acab.Date[i]),
            "Favorites": str(df_tweets_acab.Favorites[i]),
            "Retweets": str(df_tweets_acab.Retweets[i]),
            "Mentions": str(df_tweets_acab.Mentions[i]),
            "Hashtags": str(df_tweets_acab.Hashtags[i])
            }
    collection.insert_one(item)

Connected successfully!!!


In [12]:
# Safe Data in DATABASE:
MONGO_HOST= 'mongodb://localhost/'
client = MongoClient(MONGO_HOST)
try:
    connect = MongoClient()
    print("Connected successfully!!!")
except:
    print("Could not connect to MongoDB")
    
db = connect.Hashtags_all
collection = db.Final_BLM

for i in range(0,len(df_tweets_blm.User)):
    item = {
            "ID": str(df_tweets_blm.ID[i]),
            "User": str(df_tweets_blm.User[i]),
            "Text": str(df_tweets_blm.Text[i]),
            "Date": str(df_tweets_blm.Date[i]),
            "Favorites": str(df_tweets_blm.Favorites[i]),
            "Retweets": str(df_tweets_blm.Retweets[i]),
            "Mentions": str(df_tweets_blm.Mentions[i]),
            "Hashtags": str(df_tweets_blm.Hashtags[i])
            }
    collection.insert_one(item)

Connected successfully!!!


# Verificar en BD

In [5]:
#Leemos el primer set de top tweets recolectado entre el 22/05 y el 21/06 sobre #BlackLivesMatter y almacenado en MongoDB:
myclient = MongoClient()
mydb = myclient["Hashtags"]
mycol = mydb["Final_BlackLiveMatters"]
x = mycol.count_documents({})
all_tweets_BLM_top = list(mycol.find({}))
print("La cantidad de top tweets registrados para #BlackLivesMatter entre el 22/05 y el 21/06 es:",x)

La cantidad de top tweets registrados para #BlackLivesMatter entre el 22/05 y el 21/06 es: 59475


In [6]:
#Leemos el segundo set de tweets recolectado entre el 22/05 y el 21/06 sobre #BlackLivesMatter y almacenado en MongoDB:
myclient = MongoClient()
mydb = myclient["Hashtags_all"]
mycol = mydb["Final_BLM"]
x = mycol.count_documents({})
all_tweets_BLM = list(mycol.find({}))
print("La cantidad de tweets registrados para #BlackLivesMatter entre el 22/05 y el 21/06 es:",x)

La cantidad de tweets registrados para #BlackLivesMatter entre el 22/05 y el 21/06 es: 202572


In [7]:
#Leemos el primer set de tweets recolectado entre el 22/05 y el 21/06 sobre #ACAB y almacenado en MongoDB:
myclient = MongoClient()
mydb = myclient["Hashtags_all"]
mycol = mydb["Final_ACAB"]
x = mycol.count_documents({})
all_tweets_ACAB = list(mycol.find({}))
print("La cantidad de tweets registrados para #ACAB entre el 22/05 y el 21/06 es:",x)

La cantidad de tweets registrados para #ACAB entre el 22/05 y el 21/06 es: 110631


In [8]:
#Creamos un dataset del primer set de tweets de #BlackLivesMatter:
tweets = []
for result in all_tweets_BLM_top:
    tweet_dict = {}
    tweet_dict['ID'] = str(result['ID'])
    tweet_dict['User'] = str(result['User'])
    tweet_dict['Text_Original'] = str(result['Text'])
    tweet_dict["Text_Original_Size"] = len(result['Text'])
    tweet_dict['Date'] = str(result['Date'])
    tweet_dict['Favorites'] = int(result['Favorites'])
    tweet_dict['Retweets'] = int(result['Retweets'])
    tweet_dict['Mentions'] = str(result['Mentions'])
    tweet_dict['Hashtags'] = str(result['Hashtags'])
    tweets.append(tweet_dict)
df_tweets_BLM_top = pd.DataFrame(tweets)
df_tweets_BLM_top.shape

(59475, 9)

In [9]:
#Creamos un dataset del segundo set de tweets de #BlackLivesMatter:
tweets = []
for result in all_tweets_BLM:
    tweet_dict = {}
    tweet_dict['ID'] = str(result['ID'])
    tweet_dict['User'] = str(result['User'])
    tweet_dict['Text_Original'] = str(result['Text'])
    tweet_dict["Text_Original_Size"] = len(result['Text'])
    tweet_dict['Date'] = str(result['Date'])
    tweet_dict['Favorites'] = int(result['Favorites'])
    tweet_dict['Retweets'] = int(result['Retweets'])
    tweet_dict['Mentions'] = str(result['Mentions'])
    tweet_dict['Hashtags'] = str(result['Hashtags'])
    tweets.append(tweet_dict)
df_tweets_BLM = pd.DataFrame(tweets)
df_tweets_BLM.shape

(202572, 9)

In [10]:
#Creamos un dataset para el set de tweets de #ACAB:
tweets = []
for result in all_tweets_ACAB:
    tweet_dict = {}
    tweet_dict['ID'] = str(result['ID'])
    tweet_dict['User'] = str(result['User'])
    tweet_dict['Text_Original'] = str(result['Text'])
    tweet_dict["Text_Original_Size"] = len(result['Text'])
    tweet_dict['Date'] = str(result['Date'])
    tweet_dict['Favorites'] = int(result['Favorites'])
    tweet_dict['Retweets'] = int(result['Retweets'])
    tweet_dict['Mentions'] = str(result['Mentions'])
    tweet_dict['Hashtags'] = str(result['Hashtags'])
    tweets.append(tweet_dict)
df_tweets_ACAB = pd.DataFrame(tweets)
df_tweets_ACAB.shape

(110631, 9)

In [11]:
#Unimos ambos datasets sobre #BlackLivesMatter:
df_BLM = df_tweets_BLM_top.append(df_tweets_BLM)
print("La cantidad total de tweets registrados sobre #BlackLivesMatter en 30 días es:", len(df_BLM.ID))

La cantidad total de tweets registrados sobre #BlackLivesMatter en 30 días es: 262047


In [12]:
#Para el Dataset sobre #ACAB:
df_ACAB = df_tweets_ACAB
print("La cantidad de tweets registrados sobre #ACAB en 30 días es:", len(df_ACAB.ID))

La cantidad de tweets registrados sobre #ACAB en 30 días es: 110631


In [13]:
df_BLM.to_csv(r'Tweets_BLM.csv', index = False)

In [14]:
df_ACAB.to_csv(r'Tweets_ACAB.csv', index = False)